# Setting Up

In [ ]:
%matplotlib inline

In [ ]:
! pip install -q transformers[sentencepiece] fastbook fastai nbdev plum-dispatch evaluate seqeval onnxruntime onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.8/719.8 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.7/88.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 30.4 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/msi1427/blurr.git
%cd blurr

Cloning into 'blurr'...
remote: Enumerating objects: 5063, done.
remote: Counting objects: 100% (890/890), done.
remote: Compressing objects: 100% (310/310), done.
remote: Total 5063 (delta 701), reused 699 (delta 573), pack-reused 4173 (from 1)
Receiving objects: 100% (5063/5063), 27.00 MiB | 25.22 MiB/s, done.
Resolving deltas: 100% (3932/3932), done.
/content/blurr


In [ ]:
import torch
from transformers import AutoModelForSequenceClassification, AutoConfig
from fastai.text.all import *
from blurr.text.data.all import *
from blurr.text.modeling.all import *

In [ ]:
from tqdm.notebook import tqdm
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/datasets/NLP

/content/drive/MyDrive/datasets/NLP


# Data

In [ ]:
df = pd.read_csv("/content/news_multilabel_merged.csv")
df.head()

,title,text,labels,source,url,section
0,Starmer faces rebellion over plan to cut jury trials,"Nearly 40 Labour MPs have warned the prime minister they are not prepared to support proposals to limit jury trials in England and Wales. In a letter to Sir Keir Starmer, MPs largely from the left of the party said restricting juries to major offences carrying three-year terms was ""madness and will cause more problems than it solves"". Former shadow attorney general Karl Turner, who organised the letter, said he will vote against Labour for the first time since Sir Keir took charge, branding the plans ""simply unworkable"". The government insisted it would go ahead with the plans, adding they...",['Politics'],BBC,https://www.bbc.com/news/articles/c93w771g14go,NaN
1,UK to ban deepfake AI 'nudification' apps,"The UK government says it will ban so-called ""nudification"" apps as part of efforts to tackle misogyny online. New laws - announced on Thursday as part of a wider strategy to halve violence against women and girls - will make it illegal to create and supply AI tools letting users edit images to seemingly remove someone's clothing. The new offences would build on existing rules around sexually explicit deepfakes and intimate image abuse, the government said. ""Women and girls deserve to be safe online as well as offline,"" said Technology Secretary Liz Kendall. ""We will not stand by while tec...",['Politics'],BBC,https://www.bbc.com/news/articles/cq8dp2y0z7wo,NaN
2,Scotland Politics | Latest News & Updates | BBC News,The Scottish parliament's presiding officer says the incident was part of an employment dispute involving an MSP. Their union BMA Scotland has accused the government of reneging on a commitment to restore pay to 2008 levels. Opposition party leaders grilll John Swinney in the final First Minister's Questions before Christmas Why did the people of Whitburn - and nearby Blackburn - choose Nigel Farage's party over others? The delivery date for MV Glen Rosa is pushed back by up to another six months while there is a further short delay with the Turkish-built Isle of Islay. The first minister ...,['Politics'],BBC,https://www.bbc.com/news/scotland/scotland_politics,NaN
3,Foreign prison officers exempted from new stricter visa rules,"Foreign nationals working as prison officers in the UK have been given a temporary exemption from new visa rules, following warnings some jails were facing a staffing crisis. Prisons have increasingly been relying on overseas recruits, particularly from Nigeria and Ghana. But organisations representing officers had warned jails faced losing thousands of staff, after the government increased the minimum salary requirement for a skilled worker visa. The Ministry of Justice said the move would ""ensure jails can continue to run safely with the right level of experienced staff"". Under changes i...",['Politics'],BBC,https://www.bbc.com/news/articles/cp89p1x26kxo,NaN
4,Batters review: 'Bewildered' farmers need new deal to be profitable,"Farmers are ""bewildered and frightened"" with many questioning the future of their businesses because of the government's proposed changes to inheritance tax, an independent review of farm profitability has found. The long-awaited government-commissioned report was published on Thursday with 57 recommendations designed to improve productivity, investment and resilience in agriculture. But author Baroness Minette Batters, former president of the National Farmers' Union (NFU), warned there was ""no silver bullet"" to making farms in England profitable. Environment secretary Emma Reynolds said t...",['Politics'],BBC,https://www.bbc.com/news/articles/c9975z55dr5o,NaN


In [ ]:
# import ast

# # 1. Convert the 'labels' column from strings like "['a', 'b']" to actual Python lists
# # We use ast.literal_eval because it is safer than eval()
# def convert_to_list(val):
#     if isinstance(val, str):
#         try:
#             return ast.literal_eval(val)
#         except (ValueError, SyntaxError):
#             return []  # Return empty list if string is malformed
#     return val if isinstance(val, list) else []

# df['labels'] = df['labels'].apply(convert_to_list)

# # 2. Drop rows that are NOT multi-label
# # (This keeps only rows where the number of labels is 2 or more)
# df = df[df['labels'].map(len) > 1].reset_index(drop=True)

# # 3. Print the first 100 rows to verify the format
# print(f"Remaining rows after filtering: {len(df)}")

In [ ]:
df.shape

(95730, 6)

In [ ]:
df.head()

,title,text,labels,source,url,section
0,Starmer faces rebellion over plan to cut jury trials,"Nearly 40 Labour MPs have warned the prime minister they are not prepared to support proposals to limit jury trials in England and Wales. In a letter to Sir Keir Starmer, MPs largely from the left of the party said restricting juries to major offences carrying three-year terms was ""madness and will cause more problems than it solves"". Former shadow attorney general Karl Turner, who organised the letter, said he will vote against Labour for the first time since Sir Keir took charge, branding the plans ""simply unworkable"". The government insisted it would go ahead with the plans, adding they...",['Politics'],BBC,https://www.bbc.com/news/articles/c93w771g14go,NaN
1,UK to ban deepfake AI 'nudification' apps,"The UK government says it will ban so-called ""nudification"" apps as part of efforts to tackle misogyny online. New laws - announced on Thursday as part of a wider strategy to halve violence against women and girls - will make it illegal to create and supply AI tools letting users edit images to seemingly remove someone's clothing. The new offences would build on existing rules around sexually explicit deepfakes and intimate image abuse, the government said. ""Women and girls deserve to be safe online as well as offline,"" said Technology Secretary Liz Kendall. ""We will not stand by while tec...",['Politics'],BBC,https://www.bbc.com/news/articles/cq8dp2y0z7wo,NaN
2,Scotland Politics | Latest News & Updates | BBC News,The Scottish parliament's presiding officer says the incident was part of an employment dispute involving an MSP. Their union BMA Scotland has accused the government of reneging on a commitment to restore pay to 2008 levels. Opposition party leaders grilll John Swinney in the final First Minister's Questions before Christmas Why did the people of Whitburn - and nearby Blackburn - choose Nigel Farage's party over others? The delivery date for MV Glen Rosa is pushed back by up to another six months while there is a further short delay with the Turkish-built Isle of Islay. The first minister ...,['Politics'],BBC,https://www.bbc.com/news/scotland/scotland_politics,NaN
3,Foreign prison officers exempted from new stricter visa rules,"Foreign nationals working as prison officers in the UK have been given a temporary exemption from new visa rules, following warnings some jails were facing a staffing crisis. Prisons have increasingly been relying on overseas recruits, particularly from Nigeria and Ghana. But organisations representing officers had warned jails faced losing thousands of staff, after the government increased the minimum salary requirement for a skilled worker visa. The Ministry of Justice said the move would ""ensure jails can continue to run safely with the right level of experienced staff"". Under changes i...",['Politics'],BBC,https://www.bbc.com/news/articles/cp89p1x26kxo,NaN
4,Batters review: 'Bewildered' farmers need new deal to be profitable,"Farmers are ""bewildered and frightened"" with many questioning the future of their businesses because of the government's proposed changes to inheritance tax, an independent review of farm profitability has found. The long-awaited government-commissioned report was published on Thursday with 57 recommendations designed to improve productivity, investment and resilience in agriculture. But author Baroness Minette Batters, former president of the National Farmers' Union (NFU), warned there was ""no silver bullet"" to making farms in England profitable. Environment secretary Emma Reynolds said t...",['Politics'],BBC,https://www.bbc.com/news/articles/c9975z55dr5o,NaN


In [ ]:
df = df.dropna().reset_index(drop=True)
df.shape

(92330, 6)

In [ ]:
import ast
from collections import Counter

# 1. Prepare to count
label_count = Counter()

# 2. Iterate safely
for label_entry in df['labels']:
    # Convert string "[...]" to actual list safely
    if isinstance(label_entry, str):
        try:
            # We use a different variable name (current_labels) to avoid overwriting
            current_labels = ast.literal_eval(label_entry)
        except:
            continue
    else:
        current_labels = label_entry

    # Update our counter with the list of labels in this row
    if isinstance(current_labels, list):
        label_count.update(current_labels)

print(f"Number of unique labels: {len(label_count)}")
print(dict(label_count))

# 3. Calculate threshold and rare labels
threshold = int(len(df) * 0.01)
rare_labels = [key for key, value in label_count.items() if value < threshold]

print(f"Threshold (1%): {threshold}")
print(f"Number of rare labels: {len(rare_labels)}")

Number of unique labels: 12
{'Politics': 50623, 'Technology': 32667, 'Business': 19669, 'Economy': 12410, 'Sports': 11, 'International_Affairs': 10040, 'Education': 5938, 'Health': 38, 'Energy': 34, 'Crime': 10, 'Science': 4927, 'Climate': 9972}
Threshold (1%): 923
Number of rare labels: 4


In [ ]:
import ast

# 1. Convert to a list of rows
raw_labels_list = df['labels'].to_list()
revised_label_list = []
indices_to_drop = []

for idx, item in enumerate(raw_labels_list):
    # Convert string "[...]" to actual list safely, or keep if already a list
    if isinstance(item, str):
        try:
            current_row_labels = ast.literal_eval(item)
        except:
            current_row_labels = []
    else:
        current_row_labels = item

    # Filter the labels
    # Use a different name (l) to avoid shadowing the list
    revised_row = [l for l in current_row_labels if l not in rare_labels]

    # Check if any labels are left
    if len(revised_row) == 0:
        indices_to_drop.append(idx)
    else:
        revised_label_list.append(revised_row)

# 2. Update the DataFrame
df = df.drop(indices_to_drop).reset_index(drop=True)
df['revised_labels'] = revised_label_list

print(f"Original rows dropped: {len(indices_to_drop)}")
print(f"New shape: {df.shape}")

Original rows dropped: 36
New shape: (92294, 7)


In [ ]:
revised_label_list = df.revised_labels.to_list()
revised_label_count = {}
for labels in revised_label_list:
  label_list = labels
  for label in label_list:
    if label in revised_label_count.keys():
      revised_label_count[label] += 1
    else:
      revised_label_count[label] = 1
print(f"Number of labels: {len(revised_label_count)}")

Number of labels: 8


In [ ]:
encode_label_types = { key: idx for idx, (key, value) in enumerate(revised_label_count.items())}
with open("label_types_encoded-v2.json", "w") as fp:
  json.dump(encode_label_types, fp)

In [ ]:
labels = list(encode_label_types.keys())
len(labels), labels

(8,
 ['Politics',
  'Technology',
  'Business',
  'Economy',
  'International_Affairs',
  'Education',
  'Science',
  'Climate'])

In [ ]:
df.head()

,title,text,labels,source,url,section,revised_labels
0,‘It’s rather rude’: Truss accused of trying to poach members of rival Tory club,"For Tory grandees licking their wounds and plotting their return after their disastrous 2024 general election performance, the opulent, fire-lit rooms of the exclusive club 5 Hertford Street are a sanctuary. But in recent weeks, their long lunches have been rudely interrupted by Liz Truss, who has been accused of wandering the premises in search of members to poach for her own rival operation, just one street away, which asks “founding members” for an eye-watering £500,000. The former prime minister’s alleged headhunting is understood to have irritated those who run the Mayfair club, inclu...",['Politics'],Guardian,https://www.theguardian.com/politics/2025/dec/19/tory-members-club-accuses-liz-truss-of-poaching-tactics,politics,[Politics]
1,"UK politics: ‘Not clear’ who was behind FCDO hack, says minister, amid reports of China link – as it happened","We are closing this blog soon. For the latest in UK news, follow our coverage here. The UK’s Foreign, Commonwealth and Development Office was hacked in October. Details of the hack emerged on Friday in a report by the Sun that claimed a Chinese hacker group was behind the cyber-attack. The Sun named Storm 1849 as the Chinese cyber gang responsible for the breach, which it said was understood to possibly include tens of thousands of visa details. However, when asked if China was behind the attack, trade minister Chris Bryant told broadcasters it was “not clear” who perpetrated the attack an...",['Politics'],Guardian,https://www.theguardian.com/politics/live/2025/dec/19/labour-reform-conservatives-tories-lib-dems-uk-politics-latest-news-updates,politics,[Politics]
2,"UK Foreign Office victim of cyber-attack in October, says Chris Bryant","The UK’s Foreign, Commonwealth and Development Office was hacked in October, a minister has said. Chris Bryant, a trade minister in Keir Starmer’s government, told Sky News there was a low risk to “any individual” from the cyber-attack. Details of the hack emerged on Friday in a report by the Sun that claimed a Chinese hacking group was behind it. But Bryant told broadcasters it was “not clear” who perpetrated the attack and cautioned against speculation. “There certainly has been a hack at the FCDO and we’ve been aware of that since October,” Bryant told Sky News. The Sun named Storm 1849...",['Politics'],Guardian,https://www.theguardian.com/technology/2025/dec/19/uk-foreign-office-victim-cyber-attack-october-,politics,[Politics]
3,Society of Editors decries Starmer’s plan to reduce media scrutiny of No 10,"The Society of Editors has raised concerns about Keir Starmer’s plan to reduce scrutiny of No 10 by political journalists, saying it risks weakening transparency. The body, which represents news organisations, said regular, open and robust questioning was a cornerstone of democracy and that the plan to reduce briefings was deeply concerning. Downing Street’s director of communications, Tim Allan, unveiled the plan on Thursday without consulting the group of political journalists known as the lobby who traditionally attend briefings twice a day to question the prime minister’s spokesperson....",['Politics'],Guardian,https://www.theguardian.com/politics/2025/dec/19/society-of-editors-starmer-plan-reduce-lobby-briefing-media-scrutiny,politics,[Politics]
4,Reform-run Kent council accused of blocking scrutiny of claim it saved £40m,"Reform-run Kent council has been accused of trying to block scrutiny after it refused, for more than five months, to produce evidence that it had saved more than £40m by cancelling two environmental projects that did not exist yet. Polly Billington, a Labour MP in Kent, first requested background to the claim via a freedom of information (FoI) request in July. She said the subsequent delay had not been explained and seemed to show the council was embarrassed at what the documents would s

In [ ]:
import json
from collections import Counter

# Count all occurrences to build the dictionary
all_labels = [label for sublist in df.revised_labels for label in sublist]
label_counts = Counter(all_labels)

# Create the mapping: {"Politics": 0, "Technology": 1, ...}
encode_label_types = { key: idx for idx, key in enumerate(label_counts.keys())}

# Save it for later use in your model/inference
with open("label_types_encoded.json", "w") as fp:
    json.dump(encode_label_types, fp)

In [ ]:
categorical_label_list = []
num_classes = len(encode_label_types)
revised_labels_list = df.revised_labels.to_list()

for row_labels in revised_labels_list:
    # 1. Create a fresh list of zeros for THIS row
    categorical_vector = [0] * num_classes

    # 2. Fill with 1s where the label exists
    for label in row_labels:
        if label in encode_label_types:
            index = encode_label_types[label]
            categorical_vector[index] = 1

    # 3. Add this row's vector to our master list
    categorical_label_list.append(categorical_vector)

# 4. Add the new column to your dataframe
df['label_cat_list'] = categorical_label_list
print(f"Dataset shape: {df.shape}")
print(f"Encoded vector length: {len(df['label_cat_list'].iloc[0])}")

Dataset shape: (92294, 8)
Encoded vector length: 8


In [ ]:
# 1. Delete the specified columns
# We use 'errors=ignore' just in case the code is run twice
df = df.drop(columns=['genre_cat_list', 'source'], errors='ignore')

# 2. Save to CSV format
# index=False prevents pandas from adding an extra column for the row numbers
df.to_csv('cleaned_multilabel_data.csv', index=False)

# 3. Verify the final structure
print("Columns remaining:", df.columns.tolist())
print(f"File saved successfully! Final shape: {df.shape}")


Columns remaining: ['title', 'text', 'labels', 'url', 'section', 'revised_labels', 'label_cat_list']
File saved successfully! Final shape: (92294, 7)


In [ ]:
df.head()

,title,text,labels,url,section,revised_labels,label_cat_list
0,‘It’s rather rude’: Truss accused of trying to poach members of rival Tory club,"For Tory grandees licking their wounds and plotting their return after their disastrous 2024 general election performance, the opulent, fire-lit rooms of the exclusive club 5 Hertford Street are a sanctuary. But in recent weeks, their long lunches have been rudely interrupted by Liz Truss, who has been accused of wandering the premises in search of members to poach for her own rival operation, just one street away, which asks “founding members” for an eye-watering £500,000. The former prime minister’s alleged headhunting is understood to have irritated those who run the Mayfair club, inclu...",['Politics'],https://www.theguardian.com/politics/2025/dec/19/tory-members-club-accuses-liz-truss-of-poaching-tactics,politics,[Politics],"[1, 0, 0, 0, 0, 0, 0, 0]"
1,"UK politics: ‘Not clear’ who was behind FCDO hack, says minister, amid reports of China link – as it happened","We are closing this blog soon. For the latest in UK news, follow our coverage here. The UK’s Foreign, Commonwealth and Development Office was hacked in October. Details of the hack emerged on Friday in a report by the Sun that claimed a Chinese hacker group was behind the cyber-attack. The Sun named Storm 1849 as the Chinese cyber gang responsible for the breach, which it said was understood to possibly include tens of thousands of visa details. However, when asked if China was behind the attack, trade minister Chris Bryant told broadcasters it was “not clear” who perpetrated the attack an...",['Politics'],https://www.theguardian.com/politics/live/2025/dec/19/labour-reform-conservatives-tories-lib-dems-uk-politics-latest-news-updates,politics,[Politics],"[1, 0, 0, 0, 0, 0, 0, 0]"
2,"UK Foreign Office victim of cyber-attack in October, says Chris Bryant","The UK’s Foreign, Commonwealth and Development Office was hacked in October, a minister has said. Chris Bryant, a trade minister in Keir Starmer’s government, told Sky News there was a low risk to “any individual” from the cyber-attack. Details of the hack emerged on Friday in a report by the Sun that claimed a Chinese hacking group was behind it. But Bryant told broadcasters it was “not clear” who perpetrated the attack and cautioned against speculation. “There certainly has been a hack at the FCDO and we’ve been aware of that since October,” Bryant told Sky News. The Sun named Storm 1849...",['Politics'],https://www.theguardian.com/technology/2025/dec/19/uk-foreign-office-victim-cyber-attack-october-,politics,[Politics],"[1, 0, 0, 0, 0, 0, 0, 0]"
3,Society of Editors decries Starmer’s plan to reduce media scrutiny of No 10,"The Society of Editors has raised concerns about Keir Starmer’s plan to reduce scrutiny of No 10 by political journalists, saying it risks weakening transparency. The body, which represents news organisations, said regular, open and robust questioning was a cornerstone of democracy and that the plan to reduce briefings was deeply concerning. Downing Street’s director of communications, Tim Allan, unveiled the plan on Thursday without consulting the group of political journalists known as the lobby who traditionally attend briefings twice a day to question the prime minister’s spokesperson....",['Politics'],https://www.theguardian.com/politics/2025/dec/19/society-of-editors-starmer-plan-reduce-lobby-briefing-media-scrutiny,politics,[Politics],"[1, 0, 0, 0, 0, 0, 0, 0]"
4,Reform-run Kent council accused of blocking scrutiny of claim it saved £40m,"Reform-run Kent council has been accused of trying to block scrutiny after it refused, for more than five months, to produce evidence that it had saved more than £40m by cancelling two environmental projects that did not exist yet. Polly Billington, a Labour MP in Kent, first requested background to the claim via a freedom of information (FoI) request in July. She said the subsequent delay had not been explai

In [ ]:
df.shape

(92294, 7)

In [ ]:
labels = list(encode_label_types.keys())
len(labels), labels

(8,
 ['Politics',
  'Technology',
  'Business',
  'Economy',
  'International_Affairs',
  'Education',
  'Science',
  'Climate'])

# Data Split

In [ ]:
splitter = RandomSplitter(valid_pct=0.1, seed=42)
train_ids, valid_ids = splitter(df)
len(train_ids), len(valid_ids)

(83065, 9229)

In [ ]:
valid_df = df.loc[valid_ids]
valid_df.head()

,title,text,labels,url,section,revised_labels,label_cat_list
45416,"'After assembly, I cried': Surrey school grapples with race issues","The clip, which lasts just under a minute, shows a schoolgirl, blurred out to protect her identity, addressing a crowd of children – most in their school uniforms – carrying placards. Her voice echoes across the playground, shaking with emotion. “After the assembly on Tuesday, I cried. I cried out of sadness because I felt the school had invalidated my feelings and experience as a black person. I cried out of frustration because I was told that instead of standing up for myself I should stay silent,” the girl says. The speech was made in the playground of Nonsuch high school for girls in S...","['Economy', 'Education']",https://www.theguardian.com/education/2021/mar/28/after-assembly-i-cried-surrey-school-grapples-with-race-issues,education,"[Economy, Education]","[0, 0, 0, 1, 0, 1, 0, 0]"
9144,"Cop27 failed on keeping global heating to just 1.5C, Ed Miliband says – as it happened","Rishi Sunak has kept open the door to closer ties with the European Union but tried to pacify angry Brexiters in his own party by laying down a red line that the UK must remain free to set its own standards and regulation. The boss of Britain’s biggest airport group has said there is “no doubt” that Brexit has damaged the UK economy, adding that it has “massively exacerbated” worker shortages. Ed Miliband, the shadow secretary for climate change and net zero, has said Cop27 failed on the key issue of keeping global heating to just 1.5C above pre-industrial levels. (See 4.23pm.) He was spea...",['Politics'],https://www.theguardian.com/politics/live/2022/nov/21/brexit-rishi-sunak-swiss-deal-cbi-chief-uk-politics-live-latest-updates,politics,[Politics],"[1, 0, 0, 0, 0, 0, 0, 0]"
11922,Former EU diplomats urge bloc to suspend cooperation agreement with Israel,"More than 300 former European diplomats and officials have written to EU leaders urging a “far more” decisive response to the war in Gaza, including a full suspension of the bloc’s cooperation agreement with Israel. Published soon after the opening of the UN general assembly in New York, the letter from the 314 signatories also calls on all EU member states to recognise the state of Palestine, joining 147 countries to have done so. France, Belgium, the UK, Canada and Australia, among others, are expected to show support for a Palestinian state later this month at the UN. The statement foll...","['International_Affairs', 'Politics']",https://www.theguardian.com/world/2025/sep/11/former-eu-diplomats-urge-suspension-of-blocs-co-operation-agreement-with-israel,world,"[International_Affairs, Politics]","[1, 0, 0, 0, 1, 0, 0, 0]"
50441,Tour de France 2025: Tim Merlier wins stage nine as Van der Poel break falls short – as it happened,"Tour de France 2025 stage nine race report: Thank you for following the blog and getting in touch. It ended up being quite an exciting day! After a breakaway by Alpecin-Deceuninck’s Mathieu Van der Poel and Jonas Rickaert, where the duo stayed away for the majority of today’s stage, the peloton caught the attackers and the day ended in a bunch sprint. Tim Merlier of Soudal Quick-Step took his second stage win of this year’s Tour, but Van der Poel and Rickaert were successful in getting the latter on to the podium with a most combative award. Elsewhere, Elisa Longo Borghini has retained her...",['Technology'],https://www.theguardian.com/sport/live/2025/jul/13/tour-de-france-2025-stage-nine-from-chinon-to-chateauroux-live,sport,[Technology],"[0, 1, 0, 0, 0, 0, 0, 0]"
31180,"Australian-style social media ban for under-16s ‘a retrograde step’, say UK charities","Child safety experts have warned the UK government against enacting an Australian-style social media ban for children under 16, which they called a “retrograde step” that would “do more harm than good”. On Thursday, Australia became the first country in the world to ban under-16s fro

We will be using `valid_df` for all inference testing

# Fastai & Blurr Inference

In [ ]:
model_path = "models/news-final-classifier-stage.pkl"
learner_inf = load_learner(model_path)

In [ ]:
learner_inf.blurr_predict("random placeholder")

[{'labels': ['Technology'],
  'scores': [0.9305159449577332],
  'class_indices': [0, 1, 0, 0, 0, 0, 0, 0],
  'class_labels': ['Politics', 'Technology', 'Business', 'Economy', 'International_Affairs', 'Education', 'Science', 'Climate'],
  'probs': [0.04822034388780594,
   0.9305159449577332,
   0.040772438049316406,
   0.003963741939514875,
   0.04992474615573883,
   0.00906728208065033,
   0.4973832964897156,
   0.015563529916107655]}]

In [ ]:
learner_inf.blurr_predict("random placeholder")[0]['labels']

['Technology']

In [ ]:
import ast

pred = learner_inf.blurr_predict("random placeholder")[0]
labels = pred['labels']

# If it's wrapped in a string, unpack it
if len(labels) == 1 and labels[0].startswith('['):
    labels = ast.literal_eval(labels[0])

print(f"Predicted Categories: {labels}")

Predicted Categories: ['Technology']


## Evaluation

In [ ]:
from sklearn import metrics

def metric_measures(test_df, preds):

  targets = [np.asarray(target) for target in test_df['label_cat_list'].to_list()]
  outputs = [np.asarray(pred) for pred in preds]


  accuracy = metrics.accuracy_score(targets, outputs)
  f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
  f1_score_macro = metrics.f1_score(targets, outputs, average='macro')

  print(f"F1 Score (Micro) = {f1_score_micro}")
  print(f"F1 Score (Macro) = {f1_score_macro}")

  return

In [ ]:
preds = []
for idx, row in tqdm(valid_df.iterrows(), total=len(valid_df)):
  desc = row['text']
  labels = learner_inf.blurr_predict(desc)[0]['class_indices']
  # pred_genres = [0] * len(encode_genre_types)
  # for label in labels:
  #   pred_genres[encode_genre_types[label]] = 1
  preds.append(labels)

preds[0][:20]

  0%|          | 0/9229 [00:00<?, ?it/s]

[0, 1, 0, 0, 0, 1, 0, 0]

In [ ]:
# num_labels = len(encode_label_types)  # total labels
# preds = []

# for _, row in tqdm(valid_df.iterrows(), total=len(valid_df)):
#     desc = row['text']
#     label_indices = learner_inf.blurr_predict(desc)[0]['class_indices']

#     pred_vec = [0] * num_labels
#     for idx in label_indices:
#         pred_vec[idx] = 1   # directly use index

#     preds.append(pred_vec)

# preds = np.array(preds)



  0%|          | 0/9229 [00:00<?, ?it/s]

In [ ]:
num_labels = len(encode_label_types)  # = 57
print(num_labels)


8


In [ ]:
metric_measures(valid_df, preds)

F1 Score (Micro) = 0.8396309160439251
F1 Score (Macro) = 0.8253650989383114


# Convert to ONNX

### ONNX
ONNX (Open Neural Network Exchange) is a open standard for representing machine learning models.
- It allows developers to move models between different frameworks (such as PyTorch, TensorFlow, and Caffe2) without losing performance or accuracy.
ONNX makes it easier to build and run AI models on a variety of hardware, including GPUs, CPUs, and custom accelerators.
- This standard helps eliminate the need for multiple conversion tools and provides a unified representation of the model across different tools and frameworks.
- ONNX is being developed by a collaboration of companies including Microsoft, Facebook, Amazon, and IBM, among others, making it a well-supported and widely-adopted standard.
- Converting to ONNX runtime often makes model faster.

In [ ]:
model_path = "models/news-final-classifier-stage.pkl"
learner_inf = load_learner(model_path)

In [ ]:
learner_inf.model.hf_model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-5): 6 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
           

In [ ]:
!pip install onnx onnxruntime onnxscript


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 693.4/693.4 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.1/133.1 kB 12.0 MB/s eta 0:00:00


In [ ]:
classifier = learner_inf.model.hf_model.eval()

torch.onnx.export(
    classifier,
    torch.LongTensor([[0] * 512]),
    'models/news-classifier-final-version.onnx',
    verbose=True,
    input_names=['input_ids'],
    output_names=['output'],
    opset_version=14,
    dynamic_axes={
        'input_ids': {0: 'batch_size', 1: 'sequence_len'},
        'output': {0: 'batch_size'}
    }
)


W0106 06:57:55.217000 990 torch/onnx/_internal/exporter/_compat.py:114] Setting ONNX exporter to use operator set version 18 because the requested opset_version 14 is a lower version than we have implementations for. Automatic version conversion will be performed, which may not be successful at converting to the requested version. If version conversion is unsuccessful, the opset version of the exported model will be kept at 18. Please consider setting opset_version >=18 to leverage latest ONNX features


[torch.onnx] Obtain model graph for `RobertaForSequenceClassification([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `RobertaForSequenceClassification([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...
[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...


Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/onnxscript/version_converter/__init__.py", line 127, in call
    converted_proto = _c_api_utils.call_onnx_api(
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/onnxscript/version_converter/_c_api_utils.py", line 65, in call_onnx_api
    result = func(proto)
             ^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/onnxscript/version_converter/__init__.py", line 122, in _partial_convert_version
    return onnx.version_converter.convert_version(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/onnx/version_converter.py", line 39, in convert_version
    converted_model_str = C.convert_version(model_str, target_version)
                          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
RuntimeError: /github/workspace/onnx/version_converter/adapters/no_previous_version.h:26: adapt: Assertion `

[torch.onnx] Translate the graph into ONNX... ✅
Applied 21 of general pattern rewrite rules.


ONNXProgram(
    model=
        <
            ir_version=10,
            opset_imports={'': 18},
            producer_name='pytorch',
            producer_version='2.9.0+cpu',
            domain=None,
            model_version=None,
        >
        graph(
            name=main_graph,
            inputs=(
                %"input_ids"<INT64,[s72,s70]>
            ),
            outputs=(
                %"output"<FLOAT,[1,8]>
            ),
            initializers=(
                %"roberta.embeddings.token_type_embeddings.weight"<FLOAT,[1,768]>{TorchTensor(...)},
                %"roberta.embeddings.LayerNorm.weight"<FLOAT,[768]>{TorchTensor(...)},
                %"roberta.embeddings.LayerNorm.bias"<FLOAT,[768]>{TorchTensor(...)},
                %"roberta.encoder.layer.0.attention.self.query.bias"<FLOAT,[768]>{TorchTensor(...)},
                %"roberta.encoder.layer.0.attention.self.key.bias"<FLOAT,[768]>{TorchTensor(...)},
                %"roberta.encoder.layer.0.attention.sel

In [ ]:
import onnx
from onnx import shape_inference
from onnxruntime.quantization import quantize_dynamic, QuantType

# 1. Define Paths
onnx_model_path = 'models/news-classifier-final-version.onnx'
inferred_model_path = 'models/news-classifier-inferred.onnx'
quantized_model_path = 'models/news-classifier-quantized.onnx'

print("--- Starting Shape Inference ---")
try:
    # 2. Load the model
    model = onnx.load(onnx_model_path)

    # 3. Force Shape Inference
    # This populates the value_info field of the model graph,
    # which is what the quantizer needs to determine bit-depth requirements.
    inferred_model = shape_inference.infer_shapes(model)

    # 4. Save the fixed model
    onnx.save(inferred_model, inferred_model_path)
    print(f"Successfully saved inferred model to: {inferred_model_path}")

except Exception as e:
    print(f"Shape inference failed: {e}")

print("\n--- Starting Dynamic Quantization ---")
# 5. Run Quantization
# We use QInt8 (signed) as it is generally better optimized for modern CPUs
quantize_dynamic(
    model_input=inferred_model_path,
    model_output=quantized_model_path,
    weight_type=QuantType.QInt8
)

print(f"Success! Quantized model saved to: {quantized_model_path}")

# 6. Optional: Compare file sizes
import os
original_size = os.path.getsize(onnx_model_path) / (1024 * 1024)
quantized_size = os.path.getsize(quantized_model_path) / (1024 * 1024)

print(f"\nModel Size Comparison:")
print(f"Original: {original_size:.2f} MB")
print(f"Quantized: {quantized_size:.2f} MB")
print(f"Reduction: {((original_size - quantized_size) / original_size) * 100:.1f}%")

--- Starting Shape Inference ---
Successfully saved inferred model to: models/news-classifier-inferred.onnx

--- Starting Dynamic Quantization ---


InferenceError: [ShapeInferenceError] Inferred shape and existing shape differ in dimension 0: (768) vs (8)

# ONNX Inference

## Normal ONNX

In [ ]:
import onnxruntime as rt
from transformers import AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")

class_labels = list(encode_label_types.keys())

inf_session = rt.InferenceSession('models/news-classifier-final-version.onnx')
input_name = inf_session.get_inputs()[0].name
output_name = inf_session.get_outputs()[0].name

In [ ]:
preds = []
for idx, row in tqdm(valid_df.iterrows(), total=valid_df.shape[0]):
  desc = row['text']
  input_ids = tokenizer(desc)['input_ids'][:512]

  probs = inf_session.run([output_name], {input_name: [input_ids]})[0]
  probs = torch.FloatTensor(probs)

  masks = torch.sigmoid(probs) >= 0.5
  labels = [class_labels[idx] for idx, mask in enumerate(masks[0]) if mask]

  pred_genres = [0] * len(encode_label_types)
  for label in labels:
    pred_genres[encode_label_types[label]] = 1
  preds.append(pred_genres)

  0%|          | 0/9229 [00:00<?, ?it/s]

In [ ]:
metric_measures(valid_df, preds)

F1 Score (Micro) = 0.8393677053030038
F1 Score (Macro) = 0.8249254059538107


## Quantized ONNX

In [ ]:
import onnxruntime as rt
from transformers import AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")

class_labels = list(encode_genre_types.keys())

inf_session = rt.InferenceSession('models/book-classifier-quantized.onnx')
input_name = inf_session.get_inputs()[0].name
output_name = inf_session.get_outputs()[0].name

In [ ]:
preds = []
for idx, row in tqdm(valid_df.iterrows(), total=valid_df.shape[0]):
  desc = row['description']
  input_ids = tokenizer(desc)['input_ids'][:512]

  probs = inf_session.run([output_name], {input_name: [input_ids]})[0]
  probs = torch.FloatTensor(probs)

  masks = torch.sigmoid(probs) >= 0.5
  labels = [class_labels[idx] for idx, mask in enumerate(masks[0]) if mask]

  pred_genres = [0] * len(encode_genre_types)
  for label in labels:
    pred_genres[encode_genre_types[label]] = 1
  preds.append(pred_genres)

  0%|          | 0/610 [00:00<?, ?it/s]

In [ ]:
metric_measures(valid_df, preds) #

F1 Score (Micro) = 0.5283995342217622
F1 Score (Macro) = 0.16765976287838563
